In [201]:
import numpy as np
import torch as t
from torch import nn  # nn contain all of Pytorch building blocks  for neural network
t.__version__

'2.1.2+cpu'

In [202]:
what_covering = {
    1: "data (preparing and loading)",
    2: "Build model",
    3: "Fitting the model to data (training)",
    4: "Make predict and eval (inference)",
    5: "saving and loading",
    6: "Putting it all together"
}

for key in what_covering:
    print(key, ":", what_covering[key])

1 : data (preparing and loading)
2 : Build model
3 : Fitting the model to data (training)
4 : Make predict and eval (inference)
5 : saving and loading
6 : Putting it all together


In [203]:
dir(nn)

['AdaptiveAvgPool1d',
 'AdaptiveAvgPool2d',
 'AdaptiveAvgPool3d',
 'AdaptiveLogSoftmaxWithLoss',
 'AdaptiveMaxPool1d',
 'AdaptiveMaxPool2d',
 'AdaptiveMaxPool3d',
 'AlphaDropout',
 'AvgPool1d',
 'AvgPool2d',
 'AvgPool3d',
 'BCELoss',
 'BCEWithLogitsLoss',
 'BatchNorm1d',
 'BatchNorm2d',
 'BatchNorm3d',
 'Bilinear',
 'CELU',
 'CTCLoss',
 'ChannelShuffle',
 'CircularPad1d',
 'CircularPad2d',
 'CircularPad3d',
 'ConstantPad1d',
 'ConstantPad2d',
 'ConstantPad3d',
 'Container',
 'Conv1d',
 'Conv2d',
 'Conv3d',
 'ConvTranspose1d',
 'ConvTranspose2d',
 'ConvTranspose3d',
 'CosineEmbeddingLoss',
 'CosineSimilarity',
 'CrossEntropyLoss',
 'CrossMapLRN2d',
 'DataParallel',
 'Dropout',
 'Dropout1d',
 'Dropout2d',
 'Dropout3d',
 'ELU',
 'Embedding',
 'EmbeddingBag',
 'FeatureAlphaDropout',
 'Flatten',
 'Fold',
 'FractionalMaxPool2d',
 'FractionalMaxPool3d',
 'GELU',
 'GLU',
 'GRU',
 'GRUCell',
 'GaussianNLLLoss',
 'GroupNorm',
 'Hardshrink',
 'Hardsigmoid',
 'Hardswish',
 'Hardtanh',
 'HingeEmbed

## 1. Data preparing and loading
* Excel spreadsheet
* Images of any kind
* Videos
* Audio
* DNA 
* Text

## Machine learning is a game of two parts

1. Get data into numerical representation
2. Build a model to learn patterns in that numerical representation.

To showcase this, let's create some *known* data using the linear regression formula.
We'll use a linear regression formula to make a straight line with *known* parameters


In [204]:
# Create known parameters
w = 0.7  # b in LR
b = 0.3  # a in LR

# Create
start = 0
end = 1
step = 0.02
X = t.arange(start, end, step).unsqueeze(dim=1)
y = w * X + b
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [205]:
len(X), len(y)


(50, 50)

In [206]:
X.size(), y.size()

(torch.Size([50, 1]), torch.Size([50, 1]))

In [207]:
# Splitting data into training and test set
train_split = int(0.8 * len(X))  # train/test : 80/20
X_train, y_train = X[:train_split], y[:train_split]  # index 0 ~ train_split
X_test, y_test = X[train_split:], y[train_split:]  # index train_split ~ end

len(X_train), len(X_test), len(y_train), len(y_test)

(40, 10, 40, 10)

In [208]:
def visualize_data(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, predictions=None):
    fig = go.Figure()

    # Vẽ dữ liệu train
    fig.add_trace(go.Scatter(x=X_train.squeeze(), y=y_train.squeeze(), mode='markers', name='Training Data'))

    # Vẽ dữ liệu test
    fig.add_trace(go.Scatter(x=X_test.squeeze(), y=y_test.squeeze(), mode='markers', name='Test Data'))

    if predictions is not None:
        fig.add_trace(go.Scatter(x=X_test.squeeze(), y=predictions.squeeze(), mode='markers', name='Predictions'))

    fig.show()


# Gọi hàm visualize_data
visualize_data()


## Build model

In [209]:
# Create a linear regression model class
class LinearRegressionModel(nn.Module):  # almost everything in Pytorch inherits from nn.module
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(
            t.randn(1, requires_grad=True, dtype=t.float))  # randn(x): x random numbers with mean is 0
        self.bias = nn.Parameter(t.randn(1, requires_grad=True, dtype=t.float))

    # Forward
    def forward(self, x: t.Tensor) -> t.Tensor:
        return self.weights * x + self.bias




### Pytorch model building essentials
* torch.nn - contains all of the buildings for computational graphs
* torch.nn.Parameter - What parameter should our model try and learn
* torch.nn.Module - The base class for all NN module, if you subclass it, you should overwrite `forward()`
* torch.optim - this is where the optimizers live, they will help with gradient descent
* `def forward()` All nn.Module subclasses require you to overwrite `forward()`, this method defines what happens in the forward computation

### Checking the contents of our pytorch model
So we can check our model parameters of what's inside our model using `.patameters()`.

In [210]:
SEED = 42
t.manual_seed(seed=SEED)
# Create the instance of model
model_0 = LinearRegressionModel()

# Check out the parameters
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [211]:
# List the named params
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

### Making prediction using `t.inference_mode()`
To check our model's predictive power, let's see how well it predicts `y_test` based on `X_test`
When we pass data through our model, it's going to run it through the `forward()` method 

In [212]:
# make prediction with model
with t.inference_mode():  # equal t.no_grad() but t.inference is prefer
    y_preds = model_0(X_test)

visualize_data(predictions=y_preds)

### Train model

_**Things we need to train:**_
1. **Loss Function**
2. **Optimizer**
* Inside the optimizer you'll often have to set two parameters :<br>
    * `params` - The model parameters you'd like to optimize, for example `param=model_0.parameter()`
    * `lr` - The learning rate is a hyperparameter that define how big/small the optimizer changes the parameters with each step (s small `lr` results in small changes a large `lr` results in large changes)   
        * 
* _**And specifically for Pytorch**_, we need:
    * A training loop
    * a testing loop

In [213]:
# Checkout model's params
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [214]:
# Set up a loss function
loss_fn = nn.L1Loss()

# Set up an optimizer
optimizer = t.optim.SGD(
    params=model_0.parameters(),
    lr=0.01
)

### Building a training loop & Testing loop in Pytorch

__A couple of things we need in a training loop__:
1. Loop through the data
2. Forward pass (this involves data moving through our model's `forward()` functions) to make predictions on data
3. calculate the loss (compare forward pass predictions to ground truth labels)
4. Optimizer zero grad
5. Loss backward - move backwards through the network to calculate the gradient of each of parameters of our model with respect to the loss
6. Optimizer step - use the optimizer to adjust our model's parameters to try and improve the loss

In [215]:
# An epoch is one loop through the data
SEED = 27
t.manual_seed(SEED)
EPOCHS = 200

# Track diff values
epoch_count = []
loss_values = []
test_loss_values = []

### TRAINING
# 1. Loop through the data
for epoch in range(1,EPOCHS+1):
    # Set the model the training mode
    model_0.train() # train mode in Pytorch set all params that requires gradient to require gradient
    
# 2. Forward pass
    y_pred = model_0(X_train)
    
# 3. Calculate the loss
    loss = loss_fn(y_pred, y_train)
# 4. Optimizer zero grad
    optimizer.zero_grad()

# 5. Perform back propagation on the loss with respect to the parameters of the model
    loss.backward()
    
# 6. Step the optimizer (perform gradient descent)
    optimizer.step() # By default, how the optimizer changes will accumulate through the loop so we have to zero them above in step 3 to the next iteration
    
### TESTING
    model_0.eval() # turn off different settings in the model not needed for evaluation/testing (dropout/batch_normalize layers)
    with t.inference_mode():
# 1. Do the forward pass
        test_pred = model_0(X_test)
# 2. Calculate the loss
        test_loss = loss_fn(test_pred, y_test)
# 3. print
        if epoch % 10 == 0:
            epoch_count.append(epoch)
            loss_values.append(loss)
            test_loss_values.append(test_loss)
            print(f"Epoch {epoch} | Loss: {round(float(loss),3)} | test_loss: {round(float(test_loss),3)}")
            print(model_0.state_dict())
            print("")
        

Epoch 10 | Loss: 0.209 | test_loss: 0.36
OrderedDict([('weights', tensor([0.3757])), ('bias', tensor([0.2288]))])

Epoch 20 | Loss: 0.097 | test_loss: 0.228
OrderedDict([('weights', tensor([0.4146])), ('bias', tensor([0.3258]))])

Epoch 30 | Loss: 0.055 | test_loss: 0.15
OrderedDict([('weights', tensor([0.4483])), ('bias', tensor([0.3743]))])

Epoch 40 | Loss: 0.046 | test_loss: 0.115
OrderedDict([('weights', tensor([0.4728])), ('bias', tensor([0.3868]))])

Epoch 50 | Loss: 0.042 | test_loss: 0.1
OrderedDict([('weights', tensor([0.4920])), ('bias', tensor([0.3848]))])

Epoch 60 | Loss: 0.039 | test_loss: 0.089
OrderedDict([('weights', tensor([0.5100])), ('bias', tensor([0.3798]))])

Epoch 70 | Loss: 0.035 | test_loss: 0.081
OrderedDict([('weights', tensor([0.5272])), ('bias', tensor([0.3728]))])

Epoch 80 | Loss: 0.032 | test_loss: 0.073
OrderedDict([('weights', tensor([0.5441])), ('bias', tensor([0.3653]))])

Epoch 90 | Loss: 0.028 | test_loss: 0.065
OrderedDict([('weights', tensor([0

In [216]:
def visualize_history(epoch_count=epoch_count, loss_values=loss_values, test_loss_values= test_loss_values):
    # Create traces
    trace0 = go.Scatter(
        x = epoch_count,
        y = np.array(t.tensor(loss_values).numpy()),
        mode = 'lines',
        name = 'Train loss value'
    )
    
    trace1 = go.Scatter(
        x = epoch_count,
        y = test_loss_values,
        mode = 'lines',
        name = 'Test loss value'
    )
    
    data = [trace0, trace1]
    
    # Edit the layout
    layout = dict(title = 'Training and test loss curves',
                  xaxis = dict(title = 'Epochs'),
                  yaxis = dict(title = 'Loss'),
                  )
    
    fig = dict(data=data, layout=layout)
    go.Figure(fig).show()

visualize_history()

In [217]:
print(model_0.state_dict())
with t.inference_mode(): # this turn off gradient tracking & a couple more things behind the scene
    y_predicted = model_0(y_test)

    visualize_data(
        X_train=X_train,y_train=y_train,
        X_test=X_test,y_test=y_test,
        predictions=y_predicted
    )

OrderedDict([('weights', tensor([0.6990])), ('bias', tensor([0.3093]))])


### Save model in Pytorch
there are three main methods you should about for saving and loading models in Pytorch
1. `torch.save()` - save pytorch object in python's pickle (.pkl) format
2. `torch.load()` - load a saved pytorch object
3. `torch.nn.Module.load_state_dict()` - load a model's save state dictionary

### Using
1. save/load state_dict (RECOMMENDED)
`torch.save(model.state_dict(),PATH)` 
2. save/load entire model
`torch.save(model,PATH)`

In [218]:
optimizer.state_dict()

{'state': {0: {'momentum_buffer': None}, 1: {'momentum_buffer': None}},
 'param_groups': [{'lr': 0.01,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'differentiable': False,
   'params': [0, 1]}]}

In [219]:
# save model
from pathlib import Path

# 1. create model directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
# 2. create a model save path
MODEL_NAME = "01_Pytorch_workflow_model_0.pth" # .pth = .pt
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
t.save(
    obj=model_0.state_dict(),
    f=MODEL_SAVE_PATH
)

Saving model to: models\01_Pytorch_workflow_model_0.pth


### Load model
If we save model's `state_dict()` rather than entire model, we'll create a new instance of our model class and load the saved `state_dict()` into that

In [220]:
# create instance of model
loaded_model_0 = LinearRegressionModel()

# load state_dict
loaded_model_0.load_state_dict(t.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [221]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.6990])), ('bias', tensor([0.3093]))])

In [222]:
# make some prediction
loaded_model_0.eval()
with t.inference_mode():
    loaded_model_0_pred = loaded_model_0(X_test)
    model_0_pred = model_0(X_test)

# compare
loaded_model_0_pred == model_0_pred

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]])

# Putting all everything together

In [223]:
# import
import torch as t
from torch import nn
import plotly.graph_objects as go
%matplotlib inline
import matplotlib.pyplot as plt
# version
t.__version__

'2.1.2+cpu'

In [224]:
# setup device agnostic code
device = "cuda" if t.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [225]:
# Data 
w = 0.6
b = 0.4

# create range valúe
start = 0
end = 1
step = 0.02

# create X and Y
X = t.arange(start, end, step).unsqueeze(dim=1)
y = w * X + b
X, y

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800],
         [0.2000],
         [0.2200],
         [0.2400],
         [0.2600],
         [0.2800],
         [0.3000],
         [0.3200],
         [0.3400],
         [0.3600],
         [0.3800],
         [0.4000],
         [0.4200],
         [0.4400],
         [0.4600],
         [0.4800],
         [0.5000],
         [0.5200],
         [0.5400],
         [0.5600],
         [0.5800],
         [0.6000],
         [0.6200],
         [0.6400],
         [0.6600],
         [0.6800],
         [0.7000],
         [0.7200],
         [0.7400],
         [0.7600],
         [0.7800],
         [0.8000],
         [0.8200],
         [0.8400],
         [0.8600],
         [0.8800],
         [0.9000],
         [0.9200],
         [0.9400],
         [0.9600],
         [0.9800]]),
 tensor([[0.4000],
         [0.4120],
         [

In [226]:
# split data
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [227]:
# plot
visualize_data(X_train, y_train, X_test, y_test)

In [228]:
# building pytorch linear model by subclassing nn.Module

class LinearRegressionV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(
            in_features=1, # input size = 1
            out_features=1 # output size = 1
        )
        
    def forward(self, x:t.Tensor) -> t.Tensor:
        return self.linear_layer(x)
    
t.manual_seed(42)
model_1 = LinearRegressionV2()
model_1, model_1.state_dict()

(LinearRegressionV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [229]:
# check the model current device
next(model_1.parameters()).device

device(type='cpu')

In [230]:
# set the model to use the target device
model_1.to(device)
next(model_1.parameters()).device

device(type='cpu')

In [231]:
# set the data to the target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

# training
1. Loss function
2. optimizer
3. training loop
4. testing loop

In [232]:
## Training
# loss
loss_fn = nn.L1Loss() # MAE

# optimizer
optimizer = t.optim.SGD(
    lr=0.01,
    params=model_1.parameters()
)

In [233]:
# training loop
t.manual_seed(42)
epochs = 200
epoch_count = []
loss_values = []
test_loss_values = []
for epoch in range(epochs):
    model_1.train()
    
    # 1. forward pass
    y_pred = model_1(X_train)
    
    # 2. calculate the loss
    loss = loss_fn(y_pred, y_train)
    
    # 3. optimizer zero grad
    optimizer.zero_grad()
    
    # 4. backward
    loss.backward()
    
    # 5. optimizer step
    optimizer.step()
    
# testing
    model_1.eval()
    with t.inference_mode():
        test_pred = model_1(X_test)
        test_loss = loss_fn(test_pred, y_test)
        
# print 
    
    if epoch % 10 == 0 :
        print(f" Epoch: {epoch} | Loss: {loss} | Test_lost: {test_loss}")
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)

 Epoch: 0 | Loss: 0.494177907705307 | Test_lost: 0.5629762411117554
 Epoch: 10 | Loss: 0.3789680600166321 | Test_lost: 0.42826634645462036
 Epoch: 20 | Loss: 0.2637581527233124 | Test_lost: 0.2935566008090973
 Epoch: 30 | Loss: 0.14854829013347626 | Test_lost: 0.15884672105312347
 Epoch: 40 | Loss: 0.033338434994220734 | Test_lost: 0.02413688227534294
 Epoch: 50 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 60 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 70 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 80 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 90 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 100 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 110 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 120 | Loss: 0.009796462953090668 | Test_lost: 0.003305059624835849
 Epoch: 130 | Loss: 0.009796462953090668 | Test_los

In [234]:
visualize_history()
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.6007]])),
             ('linear_layer.bias', tensor([0.4095]))])

In [235]:
# Evaluate and prediction
model_1.eval()

with t.inference_mode():
    y_preds = model_1(X_test)
    visualize_data(predictions=y_preds.cpu())

In [237]:
# save
from pathlib import Path

MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True, parents=True)

MODEL_NAME = "01_Pytorch_workflow_model_1.pt"
MODEL_PATH = MODEL_DIR / MODEL_NAME

t.save(obj=model_1.state_dict(),f=MODEL_PATH)

In [238]:
# load
loaded_model_1 = LinearRegressionV2()
loaded_model_1.load_state_dict(t.load(MODEL_PATH))

<All keys matched successfully>

In [240]:
loaded_model_1.eval()
with t.inference_mode():
    loaded_model_1_pred = loaded_model_1(X_test)
    model_1_pred = model_1(X_test)
    
loaded_model_1_pred == model_1_pred

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]])